# Multi-hop Retrieval

**Multi-hop Retrieval**이란?
단일 검색으로 답을 찾을 수 없는 복합적인 질문에 대해, **단계적(Step-by-Step)으로 정보를 검색**하여 최종 답을 도출하는 기법이다.

예를 들어:
- 질문: "iPhone을 만든 회사의 본사가 위치한 도시는 어디인가?"
- 1단계(Hop 1): "iPhone을 만든 회사"를 검색 -> **Apple Inc.** 찾음
- 2단계(Hop 2): "Apple Inc.의 본사가 위치한 도시"를 검색 -> **Cupertino** 찾음
- 최종 답: **Cupertino**

In [ ]:
%pip install -Uq python-dotenv langchain langchain-openai langchain-pinecone langchain-community

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ['LANGSMITH_TRACING'] = 'true'
os.environ['LANGSMITH_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGSMITH_API_KEY'] = os.getenv('langsmith_key')
os.environ['LANGSMITH_PROJECT'] = 'skn23-langchain'
os.environ['OPENAI_API_KEY'] = os.getenv("openai_key")
os.environ['PINECONE_API_KEY'] = os.getenv("pinecone_key")
os.environ['COHERE_API_KEY'] = os.getenv('cohere_key')

## 데이터셋 로드

In [ ]:
import pandas as pd

documents_df = pd.read_csv('./documents_multihop_v2.csv')
queries_df = pd.read_csv('./queries_multihop_v2.csv')

In [ ]:
pd.set_option('display.max_colwidth', None)
documents_df

In [ ]:
queries_df

## 벡터스토어 생성 및 문서업로드

In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone() # pinecone api key 인증
print(pc.list_indexes().names())  # 현재 생성된 인덱스 목록 출력

INDEX_NAME = 'ir-multihop'
# 인덱스명 ir
if INDEX_NAME not in pc.list_indexes().names():
    pc.create_index(
        name=INDEX_NAME,
        dimension=1536,
        metric='cosine',
        spec=ServerlessSpec(
            region='us-east-1',
            cloud='aws'
        )
    )
    print(f'{INDEX_NAME}인덱스가 생성되었습니다.')
else:
    print(f'{INDEX_NAME}가 이미 존재합니다.')

In [ ]:
# DataFrame -> List[Document]
from langchain_core.documents import Document

documents = []
for idx, row in documents_df.iterrows():  # documents_df을 행별로 순회
    doc_id = row['doc_id']
    content = row['content']
    doc = Document(
        page_content=content,
        metadata={
            'doc_id': doc_id
        }
    )
    documents.append(doc)  # Document 리스트

print(len(documents))

In [ ]:
# 벡터스토어객체 생성 및 업로드
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

embedding_model = OpenAIEmbeddings(model='text-embedding-3-small')
vector_store = PineconeVectorStore(
    index_name=INDEX_NAME,
    embedding=embedding_model
) # 이미 존재하는 인덱스에 연결만
vector_store.add_documents(documents) # 문서업로드

# vector_store = PineconeVectorStore.from_documents() # 이미 존재하는 인덱스에 연결 & 문서업서트

In [ ]:
vector_store.similarity_search('손흥민이 뭐하는 사람이야?')

In [ ]:
# 벡터 서치
vector_search = vector_store.similarity_search('손흥민이 뭐하는 사람이야?')

In [ ]:
for vs in vector_search:
    print(vs.page_content)

## Multihop RAG 구현

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = init_chat_model('gpt-4.1-mini', temperature=0)
prompt = PromptTemplate.from_template('''
당신은 복잡한 질문에 답하기 위해 정보를 단계적으로 검색하는 AI에이젼트입니다.
사용자의 질문과 현재까지 수집된 정보를 바탕으로,
1. 아직 답을 찾지 못했다면:
  다음에 검색해야 할 가장 구체적이고 필요한 검색어(Query)를 출력하세요
2. 충분한 정보를 찾았다면:
  'ANSWER: '뒤에 최종 정답을 적어서 출력하세요.

### 사용자의 원래질문 ###
{original_question}

### 현재까지 수집된 정보 Context ###
{context}

### 출력지시사항 ###
불필요한 설명없이, '검색어' 또는 'ANSWER: 정답' 형식으로만 출력하세요.

1.추가검색이 필요한 경우, 검색어는 "BTS가 소속된 기획사"인 경우
("검색어" 출력하지 말것)
출력: BTS가 소속된 기획사

2.정답 추론이 가능한 경우
출력: ANSWER: 하이브
''')
output_parser = StrOutputParser()

chain = prompt | llm | output_parser

# 컨텍스트가 없는 경우
print(chain.invoke({'original_question': 'BTS가 소속된 기획사를 설립한 사람은 누구인가요?', 'context': ''}))

# 충분한 컨텍스트가 있는 경우
print(chain.invoke({'original_question': 'BTS가 소속된 기획사를 설립한 사람은 누구인가요?', 'context': '방시혁이 BTS의 기획사 하이브를 만들었다.'}))

In [ ]:
# 멀티홉 검색을 통해 단계적으로 정답을 도출하는 함수
def multihop_search(question, max_hop=3):
    context = '아직 수집된 정보 없음'
    retrieved_doc_ids = set() # 중복제거

    print(f'질문: {question}')

    for i in range(max_hop):
        print(f'\n----- {i + 1} hop -----')

        # 1.에이젼트 질의
        response = chain.invoke({'original_question': question, 'context': context})

        # 2.정답도출 여부 확인
        if response.startswith('ANSWER:'):  # ANSWER로 시작하는 경우 최종 정답
            final_answer = response.replace('ANSWER:', '').strip()  # ANSWER 제거

            print(f'\n정답: {final_answer}')
            return final_answer, retrieved_doc_ids  # 정답, 사용한 문서 ID

        # 3.검색
        query = response  # 에이전트가 제시한 다음 검색어
        docs = vector_store.similarity_search(query, k=3)  # 벡터 검색 Top-3
        # 검색문서 관리
        for doc in docs:
            retrieved_doc_ids.add(doc.metadata['doc_id'])  # 검색된 문서 ID 리스트에 추가

        # 수집된 정보를 하나의 텍스트로 병합
        content = '\n'.join([doc.page_content for doc in docs])
        print(f'\n검색어: {query}')
        print(f'검색결과:\n{content}')
        if context == '아직 수집된 정보 없음':
            context = content
        else:
            context += '\n\n' + content

    print('😥제한된 hop 수 내에 정답을 찾지 못했습니다.😥')

    return '😥답변실패😥', retrieved_doc_ids

In [ ]:
answer, retrieved_doc_ids = multihop_search('BTS가 소속된 기획사를 설립한 사람은?')
print(answer)
print(retrieved_doc_ids)

In [ ]:
answer, retrieved_doc_ids = multihop_search('영화 기생충이 상을 받은 영화제는?')
print(answer)
print(retrieved_doc_ids)

In [ ]:
answer, retrieved_doc_ids = multihop_search('아이폰 만든 회사의 본사는 어느 주에 있는가?')
print(answer)
print(retrieved_doc_ids)

In [ ]:
answer, retrieved_doc_ids = multihop_search('한국에 들어오는 테슬라 차량을 생산하는 곳은?')
print(answer)
print(retrieved_doc_ids)

## 검색 성능평가

In [ ]:
def parse_relevant_docs(relevant_str):
    """D1=1;D2=1 형태의 문자열을 파싱해서 문서ID집합을 반환"""
    return {item.split('=')[0] for item in relevant_str.split(';')}  # ';'기준으로 나눈 item을 '=' 앞의 doc_id만 추출해서 set에 담음
parse_relevant_docs('D1=1;D2=1')

In [ ]:
from tqdm.auto import tqdm

def evaluate_multihop():
    results = []

    for _, row in tqdm(queries_df.iterrows(), total=len(queries_df)):
        qid = row['query_id']
        query_text = row['query_text']
        relevant_docs = parse_relevant_docs(row['relevant_doc_ids'])  # 정답 문서 ID 집합

        # multihop수행
        answer, retrieved_docs = multihop_search(query_text)
        print('-' * 50, end='\n\n')

        # 평가 메트릭계산
        # recall = 실제 찾은 문서수 / 정답 문서수
        intersection = relevant_docs.intersection(retrieved_docs)  # 정답과 검색 결과의 교집합
        recall = len(intersection) / len(relevant_docs)

        results.append({
            'qid': qid,
            'query_text': query_text,
            'answer': answer,
            'relevant_docs': relevant_docs,
            'retrieved_docs': retrieved_docs,
            'recall': recall
        })
    return results

results = evaluate_multihop()  # 멀티홉으로 전체 평가 실행

In [ ]:
results_df = pd.DataFrame(results)
print(f'최종 recall : {results_df['recall'].mean():.4f}')

results_df

### 핵심 결과
- 멀티홉 검색은 **단일 검색으로는 해결하기 어려운 복합 질의**에서 효과적으로 동작했다.
- 질문을 단계적으로 분해하고, 각 단계에서 필요한 정보를 순차적으로 수집함으로써  
  **추론 기반 질의에 대한 Recall을 안정적으로 확보**할 수 있었다.

---

### 왜 멀티홉이 필요한가
- 단일 Dense Retrieval은 **한 문서 안에 모든 단서가 존재한다는 가정**에 의존한다.
- 실제 질의는  
  - *“A와 관련된 B는 무엇인가?”*  
  - *“A를 만든 회사의 본사는 어디인가?”*  
  와 같이 **여러 문서에 정보가 분산**된 경우가 많다.
- 멀티홉 검색은 이 문제를  
  **검색 → 추론 → 추가 검색**의 반복 구조로 해결한다.

---

### 실험을 통해 확인된 장점
- 단계별 검색으로 **정답 문서 회수율(Recall) 향상**
- 검색 과정에서 실제로 참조된 문서를 추적 가능
- LLM이 “지금 무엇을 더 찾아야 하는지”를 스스로 판단하여  
  **질의 전개(Query Decomposition)**가 자연스럽게 수행됨

---

### 한계점
- 홉(hop) 수 증가에 따라 **지연 시간과 비용 증가**
- 중간 검색이 잘못되면 이후 단계도 함께 실패할 가능성 존재
- Precision보다는 **Recall 중심 평가에 더 적합**

### 실무 적용 결론
단일 검색으로 해결 가능 → 일반 Retrieval  
복합 추론이 필요한 질문 → Multi-hop Retrieval


- 멀티홉 검색은 **지식 탐색형 QA, 리서치, 에이전트 기반 RAG**에 특히 적합
- 실제 서비스에서는  
  **Self-Query / Metadata → Dense → Multi-hop → ReRank**  
  형태로 결합하는 것이 가장 현실적인 전략이다.

---

### 한 줄 결론 (교안 / 발표용)
> **멀티홉 검색은 분산된 정보를 단계적으로 연결하여, 복합 질의에 대한 추론 가능성을 확장하는 검색 전략이다.**